In [1]:
import concurrent.futures
import polars as pl

from tools.sequence.sequence import Sequence

In [2]:
df = pl.read_csv("raw-dataset.csv")
df


source,name,sequence
str,str,str
"""antibody_monoclonal_aa_seqs.fa…","""RituximabTargetAnti-CD20v_heav…","""CAGGTGCAGCTGCAGCAGCCCGGCGCCGAG…"
"""antibody_monoclonal_aa_seqs.fa…","""RituximabTargetAnti-CD20v_ligh…","""CAGATCGTGCTGAGCCAGAGCCCCGCCATC…"
"""antibody_monoclonal_aa_seqs.fa…","""TrastuzumabTargetAnti-HER2_Lig…","""GACATCCAGATGACCCAGAGCCCCAGCAGC…"
"""antibody_monoclonal_aa_seqs.fa…","""TrastuzumabTargetAnti-HER2_Hea…","""GAGGTGCAGCTGGTGGAGAGCGGCGGCGGC…"
"""antibody_monoclonal_aa_seqs.fa…","""Bevacizumab_light_chain""","""GACATCCAGATGACCCAGAGCCCCAGCAGC…"
…,…,…
"""iedb_antigen_aa_seqs.fa""","""sp|Q8IX19|MCEM1_HUMAN Mast cel…","""ATGGAGGTGGAGGAGATCTACAAGCACCAG…"
"""iedb_antigen_aa_seqs.fa""","""sp|Q9D287|SPF27_MOUSE Pre-mRNA…","""ATGGCCGGCACCGGCCTGGTGGCCGGCGAG…"
"""iedb_antigen_aa_seqs.fa""","""tr|Q4CN05|Q4CN05_TRYCC Trans-s…","""ATGAGCAGACACCTGTTCTACAGCGCCGTG…"


In [3]:
def _mfe(s: str):
    return Sequence.from_string(s).minimum_free_energy

def _pseudo_mfe(s: str):
    return Sequence.create(s).pseudo_minimum_free_energy

def _windowed_mfe(s: str):
    return Sequence.create(s).windowed_minimum_free_energy

with concurrent.futures.ProcessPoolExecutor() as executor:
    mfes = executor.map(_mfe, df["sequence"].to_list())
    pseudo_mfes = executor.map(_pseudo_mfe, df["sequence"].to_list())
    windowed_mfes = executor.map(_windowed_mfe, df["sequence"].to_list())

df = df.with_columns(
    pl.Series("MFE", [it.energy for it in mfes]),
    pl.Series("Average MFE", [it.average_energy for it in mfes]),
    pl.Series("MFE Structure", [it.structure for it in mfes]),
    pl.Series("Pseudo-MFE", pseduo_mfes),
    pl.Series("Mean windowed MFE", [it.mean_energy for it in windowed_mfes]),
)
df

KeyboardInterrupt: 

In [7]:
df.write_csv("ensemble-ccds-with-mfe.csv")